In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Key Stations
kstations = ['07BE001', '07CD005', '07DA001', '07DD001']

# East Tributaries
etstations = ['07CD001', '07CD004', '07DA006', '07DA008', '07DC001']

# West Tributaries
wtstations = ['07DA007', '07DA018', '07DB001', '07DA032', '07DA045', '07DA033', '07DA041', '07DA040']

# Just using extracted csv files for now
hydat_data_path = r'C:\Users\LeachJ\PycharmProjects\tinihydat\HYDAT'

In [3]:
class check_highs_lows:
    def __init__(self, stalst, hydatpath):
        self.stalst = stalst
        self.hydat_path = hydatpath
        self.df_flow = self.get_data_from_csv(self.hydat_path, 'FLOW')
        
    def get_data_from_csv(self, dataFolder, parameter):
        """
        Read HYDAT *.csv files and make dataframe
        :param dataFolder: Folder with extracted *.csv files
        :param parameter: 'Flow' or 'Level'
        :return: DataFrame of parameter value time series
        """
        df = pd.DataFrame(columns=['Date'])  # Create empty dataframe for station data
        for station in self.stalst:
            if os.path.exists(os.path.join(dataFolder, station + "_Daily_" + parameter + "_ts.csv")):
                temp_df = pd.read_csv(os.path.join(dataFolder, station + "_Daily_" + parameter + "_ts.csv"),
                                      encoding='mbcs')
                # Format of data is assumed to be default pulled from HYDAT using ECDE
                temp_df = temp_df.drop([len(temp_df) - 1, len(temp_df) - 2])
                staID = temp_df[' ID'][0]  # Assumes all files have same exact headers
                temp_dff = temp_df.drop(columns=[' ID', 'PARAM', 'SYM'])  # Assumes all files have same exact headers
                temp_dff.columns = ['Date', staID]
                df = df.merge(temp_dff, how='outer', on='Date')
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values(by='Date', inplace=True)
        return df.set_index('Date')
    
    def check_low(self, year):
        print('CHECKING FOR NEW RECORD LOW FLOWS')
        df_cy = self.df_flow[self.df_flow.index.year == year]
        df_pys = self.df_flow[self.df_flow.index.year < year]
        
        tmp = 0
        tmp2 = 0
        for sta in self.stalst:
            print(sta)
            for i in range(1, 367):
                try:
                    a = df_cy[sta][df_cy[sta].index.day_of_year == i].values[0]
                    b = df_pys[sta][df_pys[sta].index.day_of_year == i].min()
                    if a < b:
                        # print(f"{df_cy[sta][df_cy[sta].index.day_of_year == i].index[0].strftime('%b %d')}")
                        tmp += 1
                except IndexError:
                    pass
            
            if tmp > 0:
                print(f"{tmp} new low flow records set in {year} for {sta}.\n")
            else:
                print(f"No new low flow records set in {year} for {sta}.\n")
            tmp2 += tmp
            tmp = 0
        
        if tmp2 > 0:
            print(f"{tmp2} new low flow records set in {year}.\n")
        else:
            print(f"No new low flow records set in {year}.\n")
    
    def check_high(self, year):
        print('CHECKING FOR NEW RECORD HIGH FLOWS')
        df_cy = self.df_flow[self.df_flow.index.year == year]
        df_pys = self.df_flow[self.df_flow.index.year < year]
        
        tmp = 0
        tmp2 = 0
        for sta in self.stalst:
            print(sta)
            for i in range(1, 367):
                try:
                    a = df_cy[sta][df_cy[sta].index.day_of_year == i].values[0]
                    b = df_pys[sta][df_pys[sta].index.day_of_year == i].max()
                    if a > b:
                        # print(f"{df_cy[sta][df_cy[sta].index.day_of_year == i].index[0].strftime('%b %d')}")
                        tmp += 1
                except IndexError:
                    pass
            
            if tmp > 0:
                print(f"{tmp} new high flow records set in {year} for {sta}.\n")
            else:
                print(f"No new high flow records set in {year} for {sta}.\n")
            tmp2 += tmp
            tmp = 0
        
        if tmp2 > 0:
            print(f"{tmp2} new high flow records set in {year}.\n")
        else:
            print(f"No new high flow records set in {year}.\n")

In [4]:
test = check_highs_lows(kstations, hydat_data_path)
test.check_low(2022)
test.check_high(2022)

CHECKING FOR NEW RECORD LOW FLOWS
07BE001
4 new low flow records set in 2022 for 07BE001.

07CD005
No new low flow records set in 2022 for 07CD005.

07DA001
No new low flow records set in 2022 for 07DA001.

07DD001
25 new low flow records set in 2022 for 07DD001.

29 new low flow records set in 2022.

CHECKING FOR NEW RECORD HIGH FLOWS
07BE001
No new high flow records set in 2022 for 07BE001.

07CD005
No new high flow records set in 2022 for 07CD005.

07DA001
No new high flow records set in 2022 for 07DA001.

07DD001
No new high flow records set in 2022 for 07DD001.

No new high flow records set in 2022.



In [5]:
test = check_highs_lows(etstations, hydat_data_path)
test.check_low(2022)
test.check_high(2022)

CHECKING FOR NEW RECORD LOW FLOWS
07CD001
1 new low flow records set in 2022 for 07CD001.

07CD004
No new low flow records set in 2022 for 07CD004.

07DA006
8 new low flow records set in 2022 for 07DA006.

07DA008
19 new low flow records set in 2022 for 07DA008.

07DC001
7 new low flow records set in 2022 for 07DC001.

35 new low flow records set in 2022.

CHECKING FOR NEW RECORD HIGH FLOWS
07CD001
No new high flow records set in 2022 for 07CD001.

07CD004
21 new high flow records set in 2022 for 07CD004.

07DA006
No new high flow records set in 2022 for 07DA006.

07DA008
No new high flow records set in 2022 for 07DA008.

07DC001
No new high flow records set in 2022 for 07DC001.

21 new high flow records set in 2022.



In [6]:
test = check_highs_lows(wtstations, hydat_data_path)
test.check_low(2022)
test.check_high(2022)

CHECKING FOR NEW RECORD LOW FLOWS
07DA007
83 new low flow records set in 2022 for 07DA007.

07DA018
No new low flow records set in 2022 for 07DA018.

07DB001
No new low flow records set in 2022 for 07DB001.

07DA032
125 new low flow records set in 2022 for 07DA032.

07DA045
68 new low flow records set in 2022 for 07DA045.

07DA033
82 new low flow records set in 2022 for 07DA033.

07DA041
158 new low flow records set in 2022 for 07DA041.

07DA040
93 new low flow records set in 2022 for 07DA040.

609 new low flow records set in 2022.

CHECKING FOR NEW RECORD HIGH FLOWS
07DA007
14 new high flow records set in 2022 for 07DA007.

07DA018
No new high flow records set in 2022 for 07DA018.

07DB001
No new high flow records set in 2022 for 07DB001.

07DA032
33 new high flow records set in 2022 for 07DA032.

07DA045
15 new high flow records set in 2022 for 07DA045.

07DA033
18 new high flow records set in 2022 for 07DA033.

07DA041
46 new high flow records set in 2022 for 07DA041.

07DA040
25 ne